# 步骤二：（仅alice和bob）数据加密

数据在进入TEE计算之前，需要对数据进行加密。

TrustedFlow提供了[CLI工具](https://github.com/secretflow/capsule-manager-sdk)来协助您完成相关操作，CLI工具提供了数据加密、授权和导出等功能。

如果您想了解更多数据加密的原理，可以参见[TrustedFlow原理](../architecture/principle.md)。

## 第一步：下载CLI工具

> 在安装CLI工具之前，您需要准备python3.8环境，建议使用conda管理您的python环境。

```bash
pip install capsule-manager-sdk
```

CLI工具详细的功能和使用方式详细参见[capsule-manager-sdk](https://github.com/secretflow/capsule-manager-sdk)，这里我们简短说明一下工具提供的命令。

- cms_config: 对工具的配置进行设置，配置文件默认保存在当前目录下的cms/cli.yaml，您也可以直接修改配置文件来完成设置。cms目录下还提供了cli-template.yaml作为模板。
- cms: 访问CapsuleManager服务。
- cms_util: 提供诸如加密、解密等功能。

## 第二步：加密数据

这里我们以开源数据集[breast cancer](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)为例，breast cancer是由加利福尼亚大学尔湾分校（UCI）提供的乳腺癌诊断数据，该数据集一共包含569个样本，每个样本拥有一个ID和10个特征，是一个典型的二分类数据集。

假设breast cancer是由alice和bob两个机构分别持有，为了方便演示，我们已经提前将breast cancer数据集拆分成了两部分，alice拥有前面5个特征，bob拥有后5个特征。拆分后的数据集可以点击链接进行下载，alice只需下载alice.csv，bob只需下载bob.csv。

[alice.csv](https://secretflow-data.oss-cn-shanghai.aliyuncs.com/datasets/breast_cancer/alice.csv)
[bob.csv](https://secretflow-data.oss-cn-shanghai.aliyuncs.com/datasets/breast_cancer/bob.csv)

1. alice生成数据密钥，用于加密数据。（该密钥请妥善保存，请勿泄露给他人）

执行后将会输出一个BASE64编码后的随机16字节的数据密钥，您也可以使用其他方式自行生成数据密钥。

```bash
cms_util generate-data-key-b64
```
2. alice加密数据文件，记得替换数据密钥为上一步得到的密钥，加密后的文件保存为alice.csv.enc。
```bash
cms_util encrypt-file --source-file alice.csv --dest-file alice.csv.enc --data-key-b64 数据密钥
```
3. bob同样的执行1和2步骤，记得bob要加密的文件是bob.csv，加密后得到的文件为bob.csv.enc。

## 第三步：上传数据密钥

数据加密后，需要把数据密钥安全的传输给CapsuleManager。CLI工具提供了对应能力。

CLI工具需要根据配置文件进行工作。我们提供了[模板配置文件](https://github.com/secretflow/capsule-manager-sdk/blob/main/python/cli/cli-template.yaml)，您可以下载到本地并放到cli/cms目录下，文件名保存为cli.yaml。

```bash
mkdir -p cli/cms
cp cli-template.yaml cli/cms/cli.yaml
```

接下来您需要对配置文件进行修改以完成数据密钥上传。

### 选项一：仿真模式

1. alice对cli.yaml进行配置，主要配置CapsuleManager的地址、是否是仿真模式（以及TLS证书，如果开启了tls模式）。
成功执行后可以在cli/cms/cli.yaml文件中看到初始化配置的内容。

- 其中`--root-ca-file`、`--private-key-file`、`--cert-chain-file`只有在CapsuleManager开启mTLS功能后才需要填写。（您也可以修改cli.yaml进行配置）
- host填写CapsuleManager的服务地址。

```bash
cms_config init --host {CapsuleManager的服务地址} --sim true --private-key-file alice.key --cert-chain-file alice.crt --root-ca-file alice_ca.crt 
```

2. alice获取自身的机构ID，机构ID是一个BASE32编码的客户端公钥的哈希值，可以用以下命令生成。
```bash
cms_util generate-party-id --cert-file alice.crt
```

3. alice配置party id、证书、私钥、加密算法等。
```bash
cms_config common --party-id xxx --scheme RSA
```

4. alice配置上传密钥信息，请在cli.yaml文件中手动配置。

- `resource_uris`：我们给加密数据取一个名称，在后面授权的时候会用到。比如，我们把这份数据叫做`breaset_cancer_alice`。这个名称后续会用到。
- `data_key_b64s`：填写上一步得到的数据加密密钥。

```yaml
register_data_keys:
  resource_uris:
    - breast_cancer_alice
  data_key_b64s: 
    - xxxx
```

5. alice上传密钥。

```bash
cms register-data-keys
```

6. bob类似地执行前面1~5步。假设bob的数据名字叫做`breast_cancer_bob`。

### 选项二：SGX模式

与仿真模式相比，SGX模式下，CLI工具会对CapsuleManager进行[远程认证](../architecture/tee/sgx.md#远程认证)，以确保CapsuleManager的代码和部署符合预期。

远程认证要求提供CapsuleManager的mrenclave（关于mrenclave的解释可以阅读[资料](../architecture/tee/sgx.md#enclave身份标识---mrenclave-和-mrsigner)），步骤一里面描述了如何获得CapsuleManager的mrencalve。

您可以先阅读仿真模式下的说明，然后根据上述说明获得mrenclave，和仿真模式的唯一差别在于需要在cli.yaml配置mrencalve，示意如下，其余步骤与仿真模式一样。

```yaml
mrenclave: "您获得的mrenclave"
```